#####  Inference kernel
* Unet Efficnet B5 
* Unet FPN Efficnet Net  B5 
* Classifcation based on Efficnet Net  B4 


In [0]:
import os 
os.listdir('../input/')


['visdom',
 'unet-efficientb5-no-aug-11-09',
 'effb4-classifer-fold4',
 'fpn-effb5-softmax-22-10',
 'segementiation-models-21-10',
 'pre-train-models',
 'efficentb4-one-classifer-steel-28-09',
 'unetmodelscript',
 'efficentnet-pytorch-0-4-0',
 'unet-seresnet50-bcediceloss-17-09',
 'efficientunet-pytorch-master',
 'jsonpatch',
 'jsonpointer',
 'pretrainedmodels',
 'torchnet',
 'torchfile',
 'severstal-steel-defect-detection']

In [0]:
!pip install ../input/pre-train-models/pretrainedmodels-0.7.4

## Impoer libraries 

In [0]:
import pdb
import os
import cv2
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset
from albumentations import (Normalize, Compose)
from albumentations.pytorch.transforms import ToTensor
import torch.utils.data as data
import torch.nn as nn
import sys


## Packages installations 

In [0]:
!pip install  ../input/efficentnet-pytorch-0-4-0/efficientnet_pytorch-0.4.0/
 

Processing /kaggle/input/efficentnet-pytorch-0-4-0/efficientnet_pytorch-0.4.0
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.4.0-cp36-none-any.whl size=11149 sha256=b049746a9159d99fe2f16cb403e801813748e67be1dbbd4d1de432afdc6aa95f
  Stored in directory: /tmp/.cache/pip/wheels/04/23/e5/4506db52c387551dc1f91890f85a32161efce173622092ffe1
Successfully built efficientnet-pytorch


In [0]:
package_path = '../input/efficientpytorch/'
sys.path.append(package_path)
from efficientnet_pytorch import EfficientNet

In [0]:
!  pip install   "../input/segementiation-models-21-10/segmentation_models.pytorch-master/segmentation_models.pytorch-master/" 

Processing /kaggle/input/segementiation-models-21-10/segmentation_models.pytorch-master/segmentation_models.pytorch-master
  Created wheel for segmentation-models-pytorch: filename=segmentation_models_pytorch-0.0.3-cp36-none-any.whl size=29146 sha256=4675211fa4c15942d01ef89966744c82c2d7322884e673019c187410703b9bd6
  Stored in directory: /tmp/pip-ephem-wheel-cache-wrerxjwe/wheels/c1/35/09/fbb62c102c3b45397fa72ef4ec30e0a91eb7c66841643e3d7a
Successfully built segmentation-models-pytorch


In [0]:
import segmentation_models_pytorch as smp

In [0]:
!pip install  ../input/efficientunet-pytorch-master/EfficientUnet-PyTorch-master/
from efficientunet import *

Processing /kaggle/input/efficientunet-pytorch-master/EfficientUnet-PyTorch-master
  Created wheel for efficientunet-pytorch: filename=efficientunet_pytorch-0.0.5-cp36-none-any.whl size=10197 sha256=f9779f0132c647509db4430c9cefcaa69459bd416609874d34e2c95de00b62c7
  Stored in directory: /tmp/.cache/pip/wheels/dc/0c/94/256e44bb61d986fb0fb17f09d52a308a328e049231d1c27dfb
Successfully built efficientunet-pytorch


## Mask to RLE

In [0]:
#https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

## Data Loader

In [0]:
class TestDataset(Dataset):
    '''Dataset for test prediction'''
    def __init__(self, root, df, mean, std):
        self.root = root
        df['ImageId'] = df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
        self.fnames = df['ImageId'].unique().tolist()
        self.num_samples = len(self.fnames)
        self.transform = Compose(
            [
                Normalize(mean=mean, std=std, p=1),
                ToTensor(),
            ]
        )

    def __getitem__(self, idx):
        fname = self.fnames[idx]
        path = os.path.join(self.root, fname)
        image = cv2.imread(path)
        images = self.transform(image=image)["image"]
        return fname, images

    def __len__(self):
        return self.num_samples

## Post Processing

In [0]:
def post_process_class(probability, threshold, classifer):
    '''Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored'''
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((256, 1600), np.float32)
    num = 0
    for c in range(1, num_component):
       p = (component == c)
       # if p.sum() > min_size:
       if classifer > 0.68: 
          predictions[p] = 1
          num += 1
    return predictions, num

In [0]:
sample_submission_path = '../input/severstal-steel-defect-detection/sample_submission.csv'
test_data_folder = "../input/severstal-steel-defect-detection/test_images"

In [0]:
# initialize test dataloader
best_threshold = 0.28
num_workers = 2
batch_size = 1
print('best_threshold', best_threshold)
min_size = 800
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
df = pd.read_csv(sample_submission_path)
testset = DataLoader(
    TestDataset(test_data_folder, df, mean, std),
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

best_threshold 0.28


## Segementation model 

In [0]:

ckpt_path = "../input/unet-efficientb5-no-aug-11-09/model.pth"
device = torch.device("cuda")
model_seg = get_efficientunet_b5(out_channels=4, concat_input=True, pretrained=False)
model_seg.to(device)
model_seg.eval()
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model_seg.load_state_dict(state["state_dict"])
for param in model_seg.parameters():
  param.requires_grad = False




ckpt_path = "../input/fpn-effb5-softmax-22-10/weight_best_FPN_full_size_fold3_1.pt"
device = torch.device("cuda")
model_seg1 = smp.FPN('efficientnet-b5', classes=4 ,encoder_weights=None,activation='softmax')
model_seg1.to(device)
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model_seg1.load_state_dict(state["state_dict"])
model_seg1.eval()

for param in model_seg1.parameters():
  param.requires_grad = False



ckpt_path = "../input/unet-seresnet50-bcediceloss-17-09/weight_best_256_fold2_14.pt"
device = torch.device("cuda")
model_seg2 = smp.Unet("se_resnet50", encoder_weights=None, classes = 4 , activation=None)
model_seg2.to(device)
model_seg2.eval()
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model_seg2.load_state_dict(state["state_dict"])

for param in model_seg2.parameters():
  param.requires_grad = False








## Classifcation Models

In [0]:
model_class = EfficientNet.from_name('efficientnet-b4') 
in_features = model_class._fc.in_features
model_class._fc = nn.Linear(in_features, 4)




ckpt_path = "../input/efficentb4-one-classifer-steel-28-09/weight_full_size__oneclassifer_fold2_10.pt"
device = torch.device("cuda")

model_class.to(device)
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model_class.load_state_dict(state["state_dict"])




model_class.eval()



for param in model_class.parameters():
  param.requires_grad = False






model_class1 = EfficientNet.from_name('efficientnet-b4') 
in_features = model_class1._fc.in_features
model_class1._fc = nn.Linear(in_features, 4)




ckpt_path = "../input/effb4-classifer-fold4/weight_full_size__oneclassifer_fold46.pt"
device = torch.device("cuda")

model_class1.to(device)
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model_class1.load_state_dict(state["state_dict"])




model_class1.eval()



for param in model_class1.parameters():
  param.requires_grad = False


In [0]:
# start prediction
min_size = [200,150,1000,1500] 
predictions = []
for i, batch in enumerate(tqdm(testset)):
    fnames, images = batch
  
    batch_preds_class  = torch.sigmoid(model_class(images.to(device))) 
    batch_preds_class = batch_preds_class.detach().cpu().numpy() 

    batch_preds_class1  = torch.sigmoid(model_class1(images.to(device))) 
    batch_preds_class1 = batch_preds_class1.detach().cpu().numpy() 

    
    batch_preds_class = (batch_preds_class + batch_preds_class1)  / 2.0



    batch_preds = torch.sigmoid(model_seg(images.to(device)))
    batch_preds = batch_preds.detach().cpu().numpy()
  
    #batch_preds2 = torch.sigmoid(model_seg1(images.to(device)))
    #batch_preds2 = batch_preds2.detach().cpu().numpy()
    
    batch_preds3 = torch.sigmoid(model_seg2(images.to(device)))
    batch_preds3 = batch_preds3.detach().cpu().numpy()
  
    batch_preds = (batch_preds +  batch_preds3) / 2.0
    #batch_preds = np.median((batch_preds,batch_preds2,batch_preds3),axis=0)  
    
    for fname, preds in zip(fnames, batch_preds):
       
        for cls, pred in enumerate(preds):
            #pred, num = post_process(pred, best_threshold, min_size[cls])
            pred, num = post_process_class(pred, best_threshold, batch_preds_class[0][cls])
            rle = mask2rle(pred)
            name = fname + f"_{cls+1}"
            predictions.append([name, rle])

# save predictions to submission.csv
df = pd.DataFrame(predictions, columns=['ImageId_ClassId', 'EncodedPixels'])
df.to_csv("submission.csv", index=False)

100%|██████████| 1801/1801 [05:40<00:00,  5.28it/s]


In [0]:
#df.head(100)

### Refrences:

 

* https://www.kaggle.com/amanooo/defect-detection-starter-u-net
* https://www.kaggle.com/go1dfish/clear-mask-visualization-and-simple-eda

A big thank you to all those who share their code on Kaggle, I'm nobody without you guys. I've learnt a lot from fellow kagglers, special shout-out to [@Abhishek](https://www.kaggle.com/abhishek), [@Yury](https://www.kaggle.com/deyury), [@Heng](https://www.kaggle.com/hengck23), [@Ekhtiar](https://www.kaggle.com/ekhtiar), [@lafoss](https://www.kaggle.com/iafoss), [@Siddhartha](https://www.kaggle.com/meaninglesslives), [@xhulu](https://www.kaggle.com/xhlulu), and the list goes on..